In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
import string
import re
from unicodedata import normalize
from matplotlib import pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
from keras.layers import Input
hidden_size=256

In [0]:
df = pd.read_csv("/content/drive/My Drive/Deep Learning/deueng/deu.txt",encoding='utf-8', delimiter = r'\t', header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
df.columns = ['English', 'German']

In [0]:
df['German'] = df['German'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [0]:
df['English']  = df['English'].str.replace("won't","will not")
df['English']  = df['English'].str.replace("ain't","am not")
df['English']  = df['English'].str.replace("'s"," is")
df['English']  = df['English'].str.replace("'m"," am")
df['English']  = df['English'].str.replace("'re'"," are")
df['English']  = df['English'].str.replace("can't","can not")
df['English']  = df['English'].str.replace("'ll"," will")
df['English']  = df['English'].str.replace("n't"," not")

In [0]:
df['German'] = df['German'].str.replace("'s"," es")

In [0]:
df['English'] = df['English'].map(lambda x: x.lower())
df['German'] = df['German'].map(lambda x: x.lower())

In [0]:
df['English'] = df['English'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x)) #removing all charecters except alphabets
df['English'] = df['English'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
df['English'] = df['English'].map(lambda x: ' '.join(word for word in x.split() if len(word)>1)) #removing single charecters'

In [0]:
df['German'] = df['German'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x)) #removing all charecters except alphabets
df['German'] = df['German'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
df['German'] = df['German'].map(lambda x: ' '.join(word for word in x.split() if len(word)>1)) #removing single charecters'

In [0]:
df['English'][22519]

'call 110 right now'

In [0]:
df.shape

(192881, 2)

In [0]:
df = df[0:10000]

In [0]:
df_train = df[:9000]
df_test = df[9000:]

In [0]:
english_tokenizer = Tokenizer()
english_tokenizer.fit_on_texts(df['English'])
eng_vocab_size = len(english_tokenizer.word_index)+1
print (eng_vocab_size)
print (max([len(x) for x in df['English'].str.split()]))

2205
5


In [0]:
eng_max_len = (max([len(x) for x in df['English'].str.split()]))

In [0]:
german_tokenizer = Tokenizer()
german_tokenizer.fit_on_texts(df['German'])
german_vocab_size = len(german_tokenizer.word_index)+1
print (german_vocab_size)
print (max([len(x) for x in df['German'].str.split()]))

3565
10


In [0]:
german_max_len = (max([len(x) for x in df['German'].str.split()]))

In [0]:
trainX = german_tokenizer.texts_to_sequences(df_train['German'])
trainX = pad_sequences(trainX, maxlen=german_max_len, padding='post')

In [0]:
trainX.shape

(9000, 10)

In [0]:
trainY = english_tokenizer.texts_to_sequences(df_train['English'])
trainY = pad_sequences(trainY, maxlen=eng_max_len, padding='post')

In [0]:
trainY.shape

(9000, 5)

In [0]:
eng_max_len = (max([len(x) for x in df['English'].str.split()]))

In [0]:
trainY = to_categorical(trainY, num_classes=eng_vocab_size)

In [0]:
trainY.shape

(9000, 5, 2205)

In [0]:
testX = german_tokenizer.texts_to_sequences(df_test['German'])
testX = pad_sequences(testX, maxlen=german_max_len, padding='post')
testY = english_tokenizer.texts_to_sequences(df_test['English'])
testY = pad_sequences(testY, maxlen=eng_max_len, padding='post')
testY = to_categorical(testY, num_classes=eng_vocab_size)
testY.shape

(1000, 5, 2205)

In [127]:
testX.shape

(1000, 10)

In [0]:
embedding_size = 256

In [0]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = tf.keras.layers.RepeatVector(german_max_len)(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = tf.keras.layers.concatenate(axis=-1,inputs = ([a,s_prev]))
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = tf.keras.layers.Dense(10, activation = "tanh")(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = tf.keras.layers.Dense(1, activation = "relu")(e) 
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = tf.keras.layers.Activation("softmax")(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = tf.keras.layers.Dot(axes = 1)([alphas,a])
    ### END CODE HERE ###
    
    return context

In [137]:
import tensorflow as tf 
from tensorflow.python.keras import Input
from tensorflow.python.keras.layers import Concatenate

"""
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(german_vocab_size,embedding_size,input_length=german_max_len))
model.add(tf.keras.layers.LSTM(embedding_size, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform'))
"""
rnn_cell_size=128

sequence_input = tf.keras.layers.Input(shape=(german_max_len,), dtype='int32')
embedded_sequences = tf.keras.layers.Embedding(german_vocab_size, 128, input_length=german_max_len)(sequence_input)
encoder_sequence_output = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.2, return_sequences=True))(embedded_sequences)


s0 = Input(shape=(128,), name='s0')
c0 = Input(shape=(128,), name='c0')
decoder_hidden_state = s0
decoder_cell_state = c0
outputs = []
for t in range(eng_max_len):
  #print(encoder_output)
  #print(decoder_hidden_state.get_shape())
  context = one_step_attention(encoder_sequence_output,decoder_hidden_state)
  #print(context.get_shape())
  decoder_hidden_state, _, decoder_cell_state = tf.keras.layers.LSTM(128, return_state = True)(context, initial_state = [decoder_hidden_state, decoder_cell_state])
  out = tf.keras.layers.Dense(eng_vocab_size, activation="softmax")(decoder_hidden_state)
  outputs.append(out)
model = tf.keras.Model(inputs = [sequence_input,s0,c0], outputs = outputs)
  
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_42 (InputLayer)           (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_40 (Embedding)        (None, 10, 128)      456320      input_42[0][0]                   
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_44 (Bidirectional (None, 10, 128)      98816       embedding_40[0][0]               
__________________________________________________________________________________________________
repeat_vec

In [0]:
opt = model.compile(optimizer=tf.keras.optimizers.Adam(decay=0.001),
                    loss='categorical_crossentropy')

In [0]:
#Declaring filepath for saving weigths
filepath = "weigths.best.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,monitor='val_acc',verbose=1,save_best_only = True, mode = 'max')

In [167]:
s0 = np.zeros((9000, 128))
c0 = np.zeros((9000, 128))
trainY_output = list(trainY.swapaxes(0,1))
model.fit([trainX, s0, c0], trainY_output, epochs=2000 ,steps_per_epoch=9, batch_size=1000, callbacks=[checkpoint], validation_data=([testX, np.zeros((1000, 128)),  np.zeros((1000, 128))], list(testY.swapaxes(0,1))))

Epoch 1/2000
9/9 [==============================] - 6s 617ms/step - loss: 15.1085 - dense_144_loss: 4.2653 - dense_147_loss: 5.3167 - dense_150_loss: 4.5574 - dense_153_loss: 0.9494 - dense_156_loss: 0.0196 - val_loss: 21.3624 - val_dense_144_loss: 5.4063 - val_dense_147_loss: 5.7633 - val_dense_150_loss: 6.8176 - val_dense_153_loss: 3.2560 - val_dense_156_loss: 0.1193
Epoch 2/2000
9/9 [==============================] - 6s 612ms/step - loss: 15.1080 - dense_144_loss: 4.2652 - dense_147_loss: 5.3167 - dense_150_loss: 4.5573 - dense_153_loss: 0.9493 - dense_156_loss: 0.0195 - val_loss: 21.3791 - val_dense_144_loss: 5.4098 - val_dense_147_loss: 5.7667 - val_dense_150_loss: 6.8242 - val_dense_153_loss: 3.2591 - val_dense_156_loss: 0.1193
Epoch 3/2000
9/9 [==============================] - 6s 614ms/step - loss: 15.1075 - dense_144_loss: 4.2651 - dense_147_loss: 5.3166 - dense_150_loss: 4.5572 - dense_153_loss: 0.9492 - dense_156_loss: 0.0194 - val_loss: 21.3953 - val_dense_144_loss: 5.4133 

KeyboardInterrupt: ignored

In [145]:
englishword_for_id = dict((v,k) for k,v in english_tokenizer.word_index.items())  
englishword_for_id[1]

'tom'

In [151]:
testX[4:5]

array([[ 11, 421,  34,   0,   0,   0,   0,   0,   0,   0]], dtype=int32)

In [165]:
for word in model.predict([pad_sequences(german_tokenizer.texts_to_sequences(["Guten Morgen Jagan"]),padding="post",maxlen=10), np.zeros((1, 128)),  np.zeros((1, 128))]):
  word_p = np.argmax(word)
  if word_p != 0:
    print (englishword_for_id[word_p])

tom
is


In [156]:
model.predict([testX[4:5], np.zeros((1, 128)),  np.zeros((1, 128))])

AttributeError: ignored

In [0]:
pad_sequences(german_tokenizer.texts_to_sequences(["Guten Morgen Jagan"]),padding="post",maxlen=10)

array([[1165,  424,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [0]:
predict_sentence_split = []
for word in np.argmax(model.predict(pad_sequences(german_tokenizer.texts_to_sequences([df['German'][9999]]),padding="post",maxlen=10)),axis=2)[0]:
  if word != 0:
    print (englishword_for_id[word])
    predict_sentence_split.append(englishword_for_id[word])

found
him


In [0]:
from nltk.translate.bleu_score import sentence_bleu

In [0]:
df['English'][9999]

'just found it'

In [0]:
sentence_bleu([df['English'][9999].split()],predict_sentence_split)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.510029457493824

In [169]:

!pip install attention_decoder


  ERROR: Could not find a version that satisfies the requirement attention_decoder (from versions: none)
ERROR: No matching distribution found for attention_decoder
